In [8]:
import pandas as pd
import re
import random
from tqdm import tqdm

path = './dataset/processed_inputs.csv'
df = pd.read_csv(path)

In [9]:
def preprocess(df_input, df_outputs):
    """
    Example Usage:
    ```
    df_input = pd.read_csv('dataset/inputs.csv')
    df_outputs = pd.read_csv('dataset/labels.csv')

    df = preprocess(df_input, df_outputs)
    ```
    """
    df = pd.merge(df_input, df_outputs, on='PatientID', how='inner')

    # Which columns to keep
    columns_to_keep = [
        'PatientID',
        'Sex',
        'HIVTesting',
        'ECigaretteUsage',
        'DifficultyConcentrating',
        'HadAsthma',
        'HadDepressiveDisorder',
        'CovidPos',
        'FluVaxLast12',
        'RaceEthnicityCategory',
        'HadDiabetes',
        'DifficultyDressingBathing',
        'ChestScan',
        'HadCOPD',
        'BlindOrVisionDifficulty',
        'HighRiskLastYear',
        'HadAngina',
        'PneumoVaxEver',
        'HadSkinCancer',
        'HadArthritis',
        'DeafOrHardOfHearing',
        'AlcoholDrinkers',
        'HadKidneyDisease',
        'TetanusLast10Tdap',
        'SmokerStatus',
        'HeightInMeters',
        'BMI',
        'HadHeartAttack'
    ]
    df = df[columns_to_keep]

    # Turn to bool
    columns_to_transform = [
        'DifficultyConcentrating',
        'HadAsthma',
        'HadDepressiveDisorder',
        'CovidPos',
        'FluVaxLast12',
        'DifficultyDressingBathing',
        'ChestScan',
        'HadCOPD',
        'BlindOrVisionDifficulty',
        'HighRiskLastYear',
        'HadAngina',
        'PneumoVaxEver',
        'HadSkinCancer',
        'HadArthritis',
        'DeafOrHardOfHearing',
        'AlcoholDrinkers',
        'HadKidneyDisease',
        'HadHeartAttack'
    ]
    df[columns_to_transform] = df[columns_to_transform].astype(bool)

    # Rounding
    df['BMI'] = df['BMI'].round(2)
    df['HeightInMeters'] = df['HeightInMeters'].round(2)

    ### Fix Column Names
    new_columns = ['Patient ID', 'Sex', 'HIV Testing', 'E-Cigarette Usage',
               'Difficulty Concentrating', 'Had Asthma', 'Had Depressive Disorder',
               'Covid Positive', 'Flu Vaccine Last 12 Months', 'Race/Ethnicity Category', 'Had Diabetes',
               'Difficulty Dressing/Bathing', 'Chest Scan', 'Had COPD',
               'Blind or Vision Difficulty', 'High Risk Last Year', 'Had Angina',
               'Pneumonia Vaccine Ever', 'Had Skin Cancer', 'Had Arthritis', 'Deaf or Hard of Hearing',
               'Alcohol Drinkers', 'Had Kidney Disease', 'Tetanus Last 10 Years (Tdap)',
               'Smoker Status', 'Height in Meters', 'BMI', 'Had Heart Attack']

    df.columns = new_columns

    return df

df_input =  pd.read_csv('./dataset/inputs.csv')
df_outputs = pd.read_csv('./dataset/labels.csv')
df = preprocess(df_input, df_outputs)

In [13]:
def format_string(row):
    # row is a row in dataframe
    feature1 = ", ".join([f"{col}: {row[col]}" for col in row.keys()])
    return feature1

def format_string_ALL(df):
    class_0 = df[df['Had Heart Attack'] == 0]
    class_1 = df[df['Had Heart Attack'] == 1]

    class_0 = class_0.sample(frac=1)
    class_1 = class_1.sample(frac=1)

    class_0 = class_0.drop(columns=["Patient ID", "Had Heart Attack"])
    class_1 = class_1.drop(columns=["Patient ID", "Had Heart Attack"])

    # class_0 and 1 are pandas df
    # Generate features for class 0 and class 1
    feature1 = [format_string(row) for _, row in class_0.sample(frac = 1).iterrows()]
    feature2 = [format_string(row) for _, row in class_1.sample(frac = 1).iterrows()]

    return feature1, feature2

In [25]:
feature0, feature1 = format_string_ALL(df)

train_index_0 = round(len(feature0)/15)
train_index_1 = round(len(feature1)/15)
test_0 = feature0[:train_index_0]
test_1 = feature1[:train_index_1]
train_0 = feature0[train_index_0:]
train_1 = feature1[train_index_1:]

In [26]:
from random import shuffle
from typing import List, Tuple

class BatchGenerator:
    def __init__(self, long_list: List, short_list: List, long_label: List, short_label: List):
        self.short_original = short_list
        self.long_original = long_list
        self.short_cycle = list(short_list)
        self.long_cycle = list(long_list)
        self.long_label = long_label
        self.short_label = short_label
        
    def get_batch(self, batch_size: int) -> List[Tuple]:
        batch = []
        
        for _ in range(batch_size):
            # Replenish and shuffle if needed
            if not self.short_cycle:
                self.short_cycle = list(self.short_original)
                shuffle(self.short_cycle)
                
            if not self.long_cycle:
                self.long_cycle = list(self.long_original)
                shuffle(self.long_cycle)
                
            # Get next items
            batch.append((self.long_cycle.pop(0), self.short_cycle.pop(0)))
        
        batch = list(zip(*batch))
        return list(batch[0]+batch[1]), self.long_label[:batch_size] + self.short_label[:batch_size]

generator = BatchGenerator(train_0, train_1, [0] * len(train_0), [1] * len(train_1))

## Transformer

In [17]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import InputExample, losses
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/scratch/xj2173/Heart-Disease-Prediction-LLM-Hackathon/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
model_Transformer = SentenceTransformer('multi-qa-mpnet-base-dot-v1').to(device)

## Regression

In [19]:
class Regression(nn.Module):
    def __init__(self):
        super(Regression, self).__init__()
        self.linear = nn.LazyLinear(1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
linear_model = Regression().to(device)

In [40]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

criterion = nn.BCELoss()
optimizer_regression = torch.optim.SGD(linear_model.parameters(), lr=0.05)
optimizer_sentence = torch.optim.Adam(model_Transformer.parameters(), lr=0.0001)

for epoch in range(1001):
    data, label = generator.get_batch(32)
    label = torch.tensor(label).to(device).unsqueeze(1).type(torch.float32)
    
    tokenized = model_Transformer.tokenize(data)
    tokenized['input_ids'] = tokenized['input_ids'].to(device)
    tokenized['attention_mask'] = tokenized['attention_mask'].to(device)

    embedding = model_Transformer(tokenized)['sentence_embedding']

    pred = linear_model(embedding)

    loss = criterion(pred,label)
    print(f'loss is {loss.item()}')

    optimizer_regression.zero_grad()
    optimizer_sentence.zero_grad()
    loss.backward()
    optimizer_regression.step()
    optimizer_sentence.step()

    if epoch % 200 == 0:
        with torch.no_grad():
            embedding = torch.tensor(model_Transformer.encode(test_0 + test_1)).to(device)
            pred = linear_model(embedding).detach().cpu().numpy()
            pred = np.round(pred)
            pred = np.asarray((pred - 1)*(-1), dtype = np.int16)
            precision, recall, f1_score, _ = precision_recall_fscore_support([1]*len(test_0) + [0]*len(test_1), pred, average='weighted')
            print(f'precision is {precision}, recall is {recall}, f1 is {f1_score}')

loss is 0.5473947525024414
precision is 0.9410798425722059, recall is 0.9380768320580579, f1 is 0.9395062181935716
loss is 0.4994422197341919
loss is 0.41477876901626587
loss is 0.6050719618797302
loss is 0.5835728049278259
loss is 0.5518221855163574
loss is 0.5557246208190918
loss is 0.5391455888748169
loss is 0.5842289924621582
loss is 0.5118210911750793
loss is 0.5117992758750916
loss is 0.49470341205596924
loss is 0.5764481425285339
loss is 0.5775840878486633
loss is 0.5201224088668823
loss is 0.4465365707874298
loss is 0.4875955283641815
loss is 0.4827383756637573
loss is 0.5841994285583496
loss is 0.5794451832771301
loss is 0.496041476726532
loss is 0.5922753810882568
loss is 0.5340266227722168
loss is 0.5470298528671265
loss is 0.5159074068069458
loss is 0.4328588843345642
loss is 0.5502130389213562
loss is 0.6166125535964966
loss is 0.5627658367156982
loss is 0.49863821268081665
loss is 0.6100817918777466
loss is 0.6729930639266968
loss is 0.5620788335800171
loss is 0.522333979

KeyboardInterrupt: 

## Scikit Logistic Regression Eval

In [25]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Split into training and testing sets
# features, labels = batch(1000)
# X = model_Transformer.encode(features)
# y = np.array(labels)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Split into training and testing sets
# features, labels = batch(2000)
# X_train, y_train = model_Transformer.encode(features), np.array(labels)  
# X_test,  y_test  = model_Transformer.encode()
# 
# 
# # Create and train the logistic regression model
# model = LogisticRegression(penalty=None)
# model.fit(X_train, y_train)
# 
# # Make predictions
# y_pred = model.predict(X_test)
# 
# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")
# print("Classification Report:")
# print(classification_report(y_test, y_pred))

Accuracy: 0.73
Classification Report:
              precision    recall  f1-score   support

         0.0       0.73      0.72      0.73       198
         1.0       0.73      0.74      0.73       202

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.73      0.73      0.73       400



/scratch/xj2173/Heart-Disease-Prediction-LLM-Hackathon/.venv/lib64/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
